In [114]:
import overpy
from geopy.geocoders import Nominatim
import pandas as pd
import numpy as np
import time

def geocode_address(address):
    geolocator = Nominatim(user_agent="your_app_name")
    location = geolocator.geocode(address)
    time.sleep(1)
    return (location.latitude, location.longitude)



def get_nearby_amenities(address):
    try:
        coordinates = geocode_address(address)

        overpass_query = f"""
            node(around:500, {coordinates[0]}, {coordinates[1]})
            ["amenity"];
            out;
        """

        api = overpy.Overpass()
        result = api.query(overpass_query)

        amenities = [node.tags.get('amenity', 'N/A') for node in result.nodes]
        time.sleep(1)
        return ", ".join(amenities)
    
    except Exception as e:
        return str(e)

def process_excel(input_file, output_file):
    df = pd.read_excel(input_file)

    df['Amenities'] = df['Address'].apply(get_nearby_amenities)

    df.to_csv(output_file, index=False)

# Example usage
input_excel_file = "final_input_addresses.xlsx"
output_csv_file = "final_output_amenities.csv"
process_excel(input_excel_file, output_csv_file)


In [143]:
import re
# Create data frame
csv = pd.read_csv("final_output_amenities.csv", encoding='utf-8')
amt = pd.DataFrame(csv)

# Extracting all categories from amenities 
amenities_mentioned = amt["Amenities"].str.split().explode().unique().tolist()
amenities_mentioned = [amenity.rstrip(",") for amenity in amenities_mentioned]

#remove irrelevant ones
remove = ["'HTTPSConnectionPool(host='nominatim.openstreetmap.org'", 'Max', 'retries', 'exceeded', 'with', 'url', 'Caused', 'by', 'Read', 'timed', 'out', 'read', "'NoneType'", 'object', 'has', 'no', 'attribute', "'latitude'", "read", "url", "Caused", "archive", "out", "HTTPSConnectionPool(host='nominatim.openstreetmap.org'"]
filtered_amenities= [amenity for amenity in amenities_mentioned if amenity not in remove]
filtered_amenities = [amenity for amenity in filtered_amenities if not any(error_word in amenity for error_word in ['Error', 'timeout', 'IncompleteRead', '/search?q='])]

# Remove non-alphabetic characters
filtered_amenities = [re.sub(r'[^a-zA-Z_]', '', amenity) for amenity in filtered_amenities]
filtered_amenities = [amenity for amenity in filtered_amenities if amenity]

#drop duplicates
amenities_mentioned = list(set(filtered_amenities))

# Print list of unique amenities
print(amenities_mentioned)

['read', 'vending_machine', 'conference_centre', 'healthcare', 'ice_cream', 'luggage_locker', 'social_facility', 'bar', 'veterinary', 'parking_space', 'gym', 'townhall', 'KR', 'police', 'pedalo', 'marketplace', 'cinema', 'office_units', 'casino', 'food_court', 'warehouse', 'bank', 'training', 'library', 'ticket_validator', 'pub', 'animal_shelter', 'toilets', 'cannabis', 'dental_hygienist', 'fountain', 'college', 'parcel_locker', 'social_centre', 'balance_beam', 'freezone', 'pharmacy', 'station', 'fire_station', 'coworking_space', 'ferry_terminal', 'Caused', 'out', 'restaurant', 'playground', 'parking', 'bicycle_parking', 'brothel', 'nursing_home', 'waste_basket', 'bench', 'place_of_worship', 'meeting_centre', 'post_office', 'school', 'community_centre', 'childcare', 'university', 'bicycle_repair_station', 'locker', 'url', 'fast_food', 'piano', 'cooking_school', 'post_box', 'waste_disposal', 'port', 'mailboxes', 'health_centre', 'bureau_de_change', 'lockers', 'courthouse', 'motorcycle_p

In [144]:
excel = "3800_reviews_english.xlsx"

# Read Excel file into DataFrame
rating_amenities = pd.read_excel(excel)

# Drop specific columns from the original DataFrame
#keep place id, overall rating, individual rating
rating_amenities = rating_amenities.drop(["name", "review_text", "location_link", "reviews", "review_datetime_utc", "reviews_per_score_1", "reviews_per_score_2", "reviews_per_score_3", "reviews_per_score_4", "reviews_per_score_5", "query", "google_id", "reviews_link", "review_id", "review_pagination_id", "author_id", "review_text_english", "review_img_url", "review_img_url", "review_questions", "review_photo_ids", "owner_answer", "owner_answer_timestamp", "review_img_urls",  "owner_answer_timestamp_datetime_utc", "review_link", "review_timestamp", "review_likes", "reviews_id"], axis=1)

# Display new DataFrame
print(rating_amenities)


                         place_id  rating  review_rating
0     ChIJCwHQg38JxkcRJeVtM9mPCnM     3.2              1
1     ChIJCwHQg38JxkcRJeVtM9mPCnM     3.2              2
2     ChIJCwHQg38JxkcRJeVtM9mPCnM     3.2              5
3     ChIJCwHQg38JxkcRJeVtM9mPCnM     3.2              1
4     ChIJCwHQg38JxkcRJeVtM9mPCnM     3.2              5
...                           ...     ...            ...
3830  ChIJGU-a6R8XxkcR5cxWLxQYpgs     4.0              4
3831  ChIJGU-a6R8XxkcR5cxWLxQYpgs     4.0              5
3832  ChIJGU-a6R8XxkcR5cxWLxQYpgs     4.0              5
3833  ChIJGU-a6R8XxkcR5cxWLxQYpgs     4.0              4
3834  ChIJGU-a6R8XxkcR5cxWLxQYpgs     4.0              5

[3835 rows x 3 columns]


In [145]:
amenity_counts_dict = {amenity: [] for amenity in amenities_mentioned}

# Loop to count the amount of amenities per location
for amenities_str in amt["Amenities"]:
    for amenity in amenities_mentioned:
        count = amenities_str.count(amenity)
        
        amenity_counts_dict[amenity].append(count)

# Add the counts as new columns in amt
for amenity, counts in amenity_counts_dict.items():
    amt[f'{amenity}_counts'] = counts

amt["total_counts"] = amt[[f"{amenity}_counts" for amenity in amenities_mentioned]].sum(axis=1)

# Display
display(amt)



/var/folders/95/3n7dglvx0g5g04d9d_rwdjq80000gn/T/ipykernel_2313/2187263265.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  amt[f'{amenity}_counts'] = counts
/var/folders/95/3n7dglvx0g5g04d9d_rwdjq80000gn/T/ipykernel_2313/2187263265.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  amt[f'{amenity}_counts'] = counts
/var/folders/95/3n7dglvx0g5g04d9d_rwdjq80000gn/T/ipykernel_2313/2187263265.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has 

,Address,place_id,Amenities,read_counts,vending_machine_counts,conference_centre_counts,healthcare_counts,ice_cream_counts,luggage_locker_counts,social_facility_counts,...,music_studio_counts,car_rental_counts,bbq_counts,kindergarten_counts,parking_entrance_counts,bicycle_rental_counts,fuel_counts,loading_dock_counts,photo_booth_counts,total_counts
0,"Julianaplein 1, Amsterdam",ChIJCwHQg38JxkcRJeVtM9mPCnM,"post_box, recycling, fast_food, restaurant, re...",0,3,0,0,0,0,0,...,0,0,0,0,2,1,0,0,0,110
1,"Joan Muyskenweg 22, 1096 CJ Amsterdam",ChIJxZHiajgLxkcRlOo3zmIibOs,"parking, post_box, parking, fuel, charging_sta...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,20
2,"Nieuwe Leeuwarderweg, 1021 BZ Amsterdam",ChIJYyOpnFQIxkcRErWFhYVuAeQ,"fuel, post_box, fast_food, pharmacy, pub, pub,...",0,0,0,0,1,0,1,...,0,0,1,0,2,0,1,0,0,121
3,"Spoorslag 29, 1082 MM Amsterdam",ChIJ15K9igQKxkcR8A7le6H02hI,"cafe, parking, place_of_worship, cafe, parking...",0,6,0,0,1,0,0,...,0,0,0,3,7,4,0,0,0,149
4,"Julianaplein Oost, 1097 DN Amsterdam",ChIJPd2JhH8JxkcRQ8jSHUchci0,"post_box, recycling, fast_food, restaurant, re...",0,3,0,0,0,0,0,...,0,0,0,0,2,1,0,0,0,121
5,"Termini 23, Amsterdam",ChIJn0fdmPwJxkcRYe7a-6JHfpI,"townhall, healthcare, social_centre, dentist, ...",0,4,0,1,0,0,1,...,0,0,0,0,4,0,0,0,0,136
6,"Stationsplein 39, 1012 AB Amsterdam",ChIJf_z8xbcJxkcRCc-nQwIMAP8,"restaurant, pub, pub, pub, restaurant, bureau_...",0,15,0,0,3,0,2,...,0,1,0,0,5,6,0,0,0,463
7,"Cornelis Lelylaan 35, 1062 HD Amsterdam",ChIJ71V5XiPixUcRSKlH5OYy-iE,"recycling, recycling, recycling, post_box, res...",0,9,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,98
8,"Europaboulevard 4A, 1083 AD Amsterdam",ChIJLWMViCIKxkcRi1ThzDUCzCo,"theatre, conference_centre, parking, parking, ...",0,4,1,0,0,0,0,...,0,2,0,0,4,0,0,0,0,97
9,"Kattenburgerstraat 5, 1018 JA Amsterdam",ChIJ8exiz50JxkcRA0dHjOHV59g,"pub, post_box, restaurant, restaurant, restaur...",0,0,0,0,0,0,0,...,0,0,0,0,3,0,0,0,0,157


In [146]:
#add place ID
place_ID_df = pd.read_excel("final_input_addresses.xlsx")

# Merge DataFrames based on adress and safe place_id
merged_df = pd.merge(amt, place_ID_df, on = "Address", how = "left")

amt["place_id"] = place_ID_df.groupby("Address")["place_id"].transform("first")
display(amt)

,Address,place_id,Amenities,read_counts,vending_machine_counts,conference_centre_counts,healthcare_counts,ice_cream_counts,luggage_locker_counts,social_facility_counts,...,music_studio_counts,car_rental_counts,bbq_counts,kindergarten_counts,parking_entrance_counts,bicycle_rental_counts,fuel_counts,loading_dock_counts,photo_booth_counts,total_counts
0,"Julianaplein 1, Amsterdam",ChIJCwHQg38JxkcRJeVtM9mPCnM,"post_box, recycling, fast_food, restaurant, re...",0,3,0,0,0,0,0,...,0,0,0,0,2,1,0,0,0,110
1,"Joan Muyskenweg 22, 1096 CJ Amsterdam",ChIJxZHiajgLxkcRlOo3zmIibOs,"parking, post_box, parking, fuel, charging_sta...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,20
2,"Nieuwe Leeuwarderweg, 1021 BZ Amsterdam",ChIJYyOpnFQIxkcRErWFhYVuAeQ,"fuel, post_box, fast_food, pharmacy, pub, pub,...",0,0,0,0,1,0,1,...,0,0,1,0,2,0,1,0,0,121
3,"Spoorslag 29, 1082 MM Amsterdam",ChIJ15K9igQKxkcR8A7le6H02hI,"cafe, parking, place_of_worship, cafe, parking...",0,6,0,0,1,0,0,...,0,0,0,3,7,4,0,0,0,149
4,"Julianaplein Oost, 1097 DN Amsterdam",ChIJPd2JhH8JxkcRQ8jSHUchci0,"post_box, recycling, fast_food, restaurant, re...",0,3,0,0,0,0,0,...,0,0,0,0,2,1,0,0,0,121
5,"Termini 23, Amsterdam",ChIJn0fdmPwJxkcRYe7a-6JHfpI,"townhall, healthcare, social_centre, dentist, ...",0,4,0,1,0,0,1,...,0,0,0,0,4,0,0,0,0,136
6,"Stationsplein 39, 1012 AB Amsterdam",ChIJf_z8xbcJxkcRCc-nQwIMAP8,"restaurant, pub, pub, pub, restaurant, bureau_...",0,15,0,0,3,0,2,...,0,1,0,0,5,6,0,0,0,463
7,"Cornelis Lelylaan 35, 1062 HD Amsterdam",ChIJ71V5XiPixUcRSKlH5OYy-iE,"recycling, recycling, recycling, post_box, res...",0,9,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,98
8,"Europaboulevard 4A, 1083 AD Amsterdam",ChIJLWMViCIKxkcRi1ThzDUCzCo,"theatre, conference_centre, parking, parking, ...",0,4,1,0,0,0,0,...,0,2,0,0,4,0,0,0,0,97
9,"Kattenburgerstraat 5, 1018 JA Amsterdam",ChIJ8exiz50JxkcRA0dHjOHV59g,"pub, post_box, restaurant, restaurant, restaur...",0,0,0,0,0,0,0,...,0,0,0,0,3,0,0,0,0,157


In [148]:
#merge the two to have ratings and amenities in one DataFrame
rating_and_count = pd.merge(amt, rating_amenities, on = "place_id", how = "left")

columns_drop = ["read_counts", "Caused_counts", "out_counts", "url_counts" ]
rating_and_count = rating_and_count.drop(columns = columns_drop)

#turn index column into 
rating_and_count.reset_index(inplace = True)

#Display merged DataFrame
display(rating_and_count)

#Save to csv
rating_and_count.to_csv("rating_and_count.csv", index = False)
for col in rating_and_count.columns:
    print(col)


,index,Address,place_id,Amenities,vending_machine_counts,conference_centre_counts,healthcare_counts,ice_cream_counts,luggage_locker_counts,social_facility_counts,...,bbq_counts,kindergarten_counts,parking_entrance_counts,bicycle_rental_counts,fuel_counts,loading_dock_counts,photo_booth_counts,total_counts,rating,review_rating
0,0,"Julianaplein 1, Amsterdam",ChIJCwHQg38JxkcRJeVtM9mPCnM,"post_box, recycling, fast_food, restaurant, re...",3,0,0,0,0,0,...,0,0,2,1,0,0,0,110,3.2,1.0
1,1,"Julianaplein 1, Amsterdam",ChIJCwHQg38JxkcRJeVtM9mPCnM,"post_box, recycling, fast_food, restaurant, re...",3,0,0,0,0,0,...,0,0,2,1,0,0,0,110,3.2,2.0
2,2,"Julianaplein 1, Amsterdam",ChIJCwHQg38JxkcRJeVtM9mPCnM,"post_box, recycling, fast_food, restaurant, re...",3,0,0,0,0,0,...,0,0,2,1,0,0,0,110,3.2,5.0
3,3,"Julianaplein 1, Amsterdam",ChIJCwHQg38JxkcRJeVtM9mPCnM,"post_box, recycling, fast_food, restaurant, re...",3,0,0,0,0,0,...,0,0,2,1,0,0,0,110,3.2,1.0
4,4,"Julianaplein 1, Amsterdam",ChIJCwHQg38JxkcRJeVtM9mPCnM,"post_box, recycling, fast_food, restaurant, re...",3,0,0,0,0,0,...,0,0,2,1,0,0,0,110,3.2,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3839,3839,"Busplein 16, 1315 KR Almere",ChIJGU-a6R8XxkcR5cxWLxQYpgs,"parking, post_box, fast_food, fast_food, fast_...",5,0,0,1,0,0,...,0,0,3,0,0,0,0,95,4.0,4.0
3840,3840,"Busplein 16, 1315 KR Almere",ChIJGU-a6R8XxkcR5cxWLxQYpgs,"parking, post_box, fast_food, fast_food, fast_...",5,0,0,1,0,0,...,0,0,3,0,0,0,0,95,4.0,5.0
3841,3841,"Busplein 16, 1315 KR Almere",ChIJGU-a6R8XxkcR5cxWLxQYpgs,"parking, post_box, fast_food, fast_food, fast_...",5,0,0,1,0,0,...,0,0,3,0,0,0,0,95,4.0,5.0
3842,3842,"Busplein 16, 1315 KR Almere",ChIJGU-a6R8XxkcR5cxWLxQYpgs,"parking, post_box, fast_food, fast_food, fast_...",5,0,0,1,0,0,...,0,0,3,0,0,0,0,95,4.0,4.0


index
Address
place_id
Amenities
vending_machine_counts
conference_centre_counts
healthcare_counts
ice_cream_counts
luggage_locker_counts
social_facility_counts
bar_counts
veterinary_counts
parking_space_counts
gym_counts
townhall_counts
KR_counts
police_counts
pedalo_counts
marketplace_counts
cinema_counts
office_units_counts
casino_counts
food_court_counts
warehouse_counts
bank_counts
training_counts
library_counts
ticket_validator_counts
pub_counts
animal_shelter_counts
toilets_counts
cannabis_counts
dental_hygienist_counts
fountain_counts
college_counts
parcel_locker_counts
social_centre_counts
balance_beam_counts
freezone_counts
pharmacy_counts
station_counts
fire_station_counts
coworking_space_counts
ferry_terminal_counts
restaurant_counts
playground_counts
parking_counts
bicycle_parking_counts
brothel_counts
nursing_home_counts
waste_basket_counts
bench_counts
place_of_worship_counts
meeting_centre_counts
post_office_counts
school_counts
community_centre_counts
childcare_counts
